In [1]:
# Calling all the libraries that we are going to use here #
library(dplyr)
library(Boruta)
library(partykit)
library(caret)
library(party)

Warning message:
"package 'dplyr' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: ranger
Warning message:
"package 'partykit' was built under R version 3.5.2"Loading required package: grid
Loading required package: libcoin
Loading required package: mvtnorm
Warning message:
"package 'caret' was built under R version 3.5.2"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'party' was built under R version 3.5.2"Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich

Attaching package: 'party'

The following objects

In [2]:
# Importing the DataSet "realestate.derived" #
re.imp<-read.csv("F:\\melb.realstate.all.files\\InputDatasets\\realestate.derived.csv", header = T ,stringsAsFactors = T, sep = ',', strip.white = TRUE, na.strings = "")
## Excluding Address, Date, Latt, Longt & Price.bucket ##
re.imp<-re.imp[,-c(2,8,18,19,22)] 


In [3]:
# Coverting var into categoricals #
re.imp$Rooms<-as.factor(re.imp$Rooms)
re.imp$Type<-as.factor(re.imp$Type)
re.imp$Bedroom2<-as.factor(re.imp$Bedroom2)
re.imp$Bathroom<-as.factor(re.imp$Bathroom)
re.imp$Car<-as.factor(re.imp$Car)
re.imp$YearBuilt<-as.factor(re.imp$YearBuilt)


In [5]:
# Importing Dataset "realestate.derived.without.year"
re.imp.noyear<-read.csv("F:\\melb.realstate.all.files\\InputDatasets\\realestate.derived.without.year.csv", header = T ,stringsAsFactors = T, sep = ',', strip.white = TRUE, na.strings = "")
## Excluding Address, Date, Latt, Longt & Price.bucket ##
re.imp.noyear<-re.imp.noyear[,-c(2,8,17,18,21)] 


In [6]:
# Coverting var into categoricals #
re.imp.noyear$Rooms<-as.factor(re.imp.noyear$Rooms)
re.imp.noyear$Type<-as.factor(re.imp.noyear$Type)
re.imp.noyear$Bedroom2<-as.factor(re.imp.noyear$Bedroom2)
re.imp.noyear$Bathroom<-as.factor(re.imp.noyear$Bathroom)
re.imp.noyear$Car<-as.factor(re.imp.noyear$Car)

In [7]:
# Applying Boruta on with year Dataset #
boruta.re.imp<-Boruta(Type~.,data = re.imp,doTrace=2)
print(boruta.re.imp)

 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +59 secs: 
 confirmed 15 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 10 more;
 rejected 1 attribute: Months;
 still have 1 attribute left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 24. run of importance source...
 25. run of importance source...


Boruta performed 28 iterations in 2.480156 mins.
 15 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 10 more;
 2 attributes confirmed unimportant: Method, Months;


In [9]:
# Applying Boruta on without year Dataset # 
boruta.re.imp.noyear<-Boruta(Type~.,data = re.imp.noyear,doTrace=2)
print(boruta.re.imp.noyear)

 1. run of importance source...
 2. run of importance source...
 3. run of importance source...
 4. run of importance source...
 5. run of importance source...
 6. run of importance source...
 7. run of importance source...
 8. run of importance source...
 9. run of importance source...
 10. run of importance source...
 11. run of importance source...
After 11 iterations, +1.9 mins: 
 confirmed 15 attributes: Bathroom, Bedroom2, BuildingArea, Car, CouncilArea and 10 more;
 still have 1 attribute left.

 12. run of importance source...
 13. run of importance source...
 14. run of importance source...
 15. run of importance source...
 16. run of importance source...
 17. run of importance source...
 18. run of importance source...
 19. run of importance source...
 20. run of importance source...
 21. run of importance source...
 22. run of importance source...
 23. run of importance source...
 24. run of importance source...
 25. run of importance source...
 26. run of importance source.

Boruta performed 70 iterations in 12.22927 mins.
 15 attributes confirmed important: Bathroom, Bedroom2, BuildingArea,
Car, CouncilArea and 10 more;
 1 attributes confirmed unimportant: Months;


In [10]:
## Removing Months due to no significance on Type ##, Method is also there need to ask abhilash??????????
re.imp<-re.imp[,-18]
## Coverting Type into numeric to perform Cforest as it won't work for categorical var ##
re.imp$Type<-as.numeric((re.imp$Type))


In [12]:
## Applying Cforest on with year Dataset ##
re.imp.rf<-party::cforest(Type~. ,data = re.imp, control=cforest_unbiased(mtry=16,ntree=200))
impvars.re.imp<-varImp(re.imp.rf)

In [13]:
## Removing Months due to no significance on Type in noyear dataset ##
re.imp.noyear<-re.imp.noyear[,-17]
## Coverting Type into numeric to perform Cforest as it won't work for categorical var ##
re.imp.noyear$Type<-as.numeric((re.imp.noyear$Type))


In [14]:
## Applying Cforext on without year Dataset ##
re.imp.noyear.rf<-party::cforest(Type~. ,data = re.imp.noyear, control=cforest_unbiased(mtry=15,ntree=200))
impvars.re.imp.noyear<-varImp(re.imp.noyear.rf)

In [16]:
print(impvars.re.imp)

                  Overall
Suburb       0.0072240599
Rooms        0.0811592150
Price        0.0497403942
Method       0.0002773736
SellerG      0.0027457668
Distance     0.0110798505
Postcode     0.0136725025
Bedroom2     0.0004855075
Bathroom     0.0002115330
Car          0.0023203637
Landsize     0.0543711373
BuildingArea 0.0747819638
YearBuilt    0.2150655310
CouncilArea  0.0124388873
Days.diff    0.0022295001
Dist.bucket  0.0054649942


In [17]:
print(impvars.re.imp.noyear)

                  Overall
Suburb       7.929975e-02
Rooms        9.569120e-02
Price        1.322236e-01
Method       2.164902e-06
SellerG      1.923561e-03
Distance     1.487230e-02
Postcode     1.012932e-02
Bedroom2     1.767188e-03
Bathroom     2.587204e-02
Car          5.658314e-03
Landsize     5.491841e-02
BuildingArea 1.166080e-01
CouncilArea  1.660685e-02
Days.diff    2.467161e-03
Dist.bucket  5.125951e-03


In [18]:
write.table(impvars.re.imp, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForType_withyear.csv",row.names=T, na="",col.names=T, sep=",")


In [19]:
write.table(impvars.re.imp.noyear, file = "F:\\melb.realstate.all.files\\OutputDataSets\\ImpVarsForType_withoutyear.csv",row.names=T, na="",col.names=T, sep=",")
